## Bestimmung des dominanten Runway-Concepts


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# ================================
# 1) Pfade & Zeitraum
# ================================
P_ARR = Path("zhaw_export_arrivals.csv")
P_DEP = Path("zhaw_export_departures.csv")
OUT_DOMINANT = Path("runway_concept_dominant_HALBHOUR.csv")

TIME_START = pd.Timestamp("2023-01-01 05:50")
TIME_STOP  = pd.Timestamp("2025-09-12 23:20")

# ================================
# 2) Helper-Funktionen
# ================================
def read_semicolon_csv(p: Path):
    df = pd.read_csv(p, sep=";", encoding="utf-8", low_memory=False)
    if df.shape[1] == 1:
        df = pd.read_csv(p, sep=";", encoding="latin1", low_memory=False)
    df.columns = df.columns.str.strip().str.lower()
    return df

def parse_time_col(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.replace(r"\s+", " ", regex=True).str.strip()
    return pd.to_datetime(s, dayfirst=True, errors="coerce")

def get_interval_start(t: pd.Timestamp) -> pd.Timestamp:
    """xx:20–xx:49 → :20, xx:50–xx:59 → :50, xx:00–xx:19 → vorherige Stunde :50."""
    if pd.isna(t):
        return pd.NaT
    m = t.minute
    if 20 <= m < 50:
        return t.replace(minute=20, second=0, microsecond=0)
    elif m >= 50:
        return t.replace(minute=50, second=0, microsecond=0)
    else:
        return (t - pd.Timedelta(hours=1)).replace(minute=50, second=0, microsecond=0)

def prep_side(df: pd.DataFrame, time_col: str, label: str) -> pd.DataFrame:
    """pro (registration, interval) 1 Zeile; runway_concept bereinigt."""
    df = df.copy()
    df[time_col] = parse_time_col(df[time_col])
    df = df[(df[time_col] >= TIME_START) & (df[time_col] <= TIME_STOP)]
    df["interval_start"] = df[time_col].apply(get_interval_start)
    df["interval_end"]   = df["interval_start"] + pd.Timedelta(minutes=30)

    if "aircraft_registration" not in df.columns:
        df["aircraft_registration"] = df.get("flight_identifier", np.nan)

    if "runway_concept" not in df.columns:
        raise ValueError(f"In den {label}-Daten fehlt 'runway_concept'.")

    df["runway_concept"] = df["runway_concept"].astype(str).str.strip().str.upper()
    df.loc[df["runway_concept"] == "", "runway_concept"] = np.nan

    # Codeshares/Mehrfachzeilen raus: pro Flugzeug & Intervall nur 1x
    df = df.drop_duplicates(subset=["aircraft_registration", "interval_start"])
    return df

# ================================
# 3) Einlesen & Bereinigen
# ================================
arr = read_semicolon_csv(P_ARR)
dep = read_semicolon_csv(P_DEP)

req_arr = "actual_landing_time_utc"
req_dep = "actual_take_off_time_utc"
if req_arr not in arr.columns:
    raise ValueError(f"In den Arrivals fehlt '{req_arr}'.")
if req_dep not in dep.columns:
    raise ValueError(f"In den Departures fehlt '{req_dep}'.")

arr_u = prep_side(arr, req_arr, "ARR")
dep_u = prep_side(dep, req_dep, "DEP")

# ================================
# 4) Konzept je Flugzeug+Intervall bestimmen
# ================================
all_flights = pd.concat([arr_u, dep_u], ignore_index=True)

# Falls ein Flugzeug im selben Intervall mehrere Konzepte hat → meistgenutztes wählen
concept_per_flight_int = (all_flights
    .groupby(["aircraft_registration", "interval_start", "runway_concept"], as_index=False)
    .size()
    .rename(columns={"size": "n"})
)
idxmax = concept_per_flight_int.groupby(["aircraft_registration", "interval_start"])["n"].idxmax()
flight_concepts = concept_per_flight_int.loc[idxmax, ["aircraft_registration", "interval_start", "runway_concept"]].copy()
flight_concepts["interval_end"] = flight_concepts["interval_start"] + pd.Timedelta(minutes=30)

# ================================
# 5) Dominantes Konzept + Gesamtzahl der betrachteten Flüge je Intervall
#    - dominant_runway_concept: Konzept mit meisten Flügen im Intervall
#    - dominant_runway_count:   *Gesamtzahl* der Flüge, die in die Dominanzwahl eingeflossen sind
# ================================
# Counts pro Intervall & Konzept
concept_counts = (
    flight_concepts.groupby(["interval_start", "interval_end", "runway_concept"], as_index=False)
    .agg(n_flights=("aircraft_registration", "nunique"))
)

# Dominantes Konzept wählen
idx_dom = concept_counts.groupby(["interval_start", "interval_end"])["n_flights"].idxmax()
dominant = concept_counts.loc[idx_dom, ["interval_start", "interval_end", "runway_concept"]].copy()
dominant = dominant.rename(columns={"runway_concept": "dominant_runway_concept"})

# *** NEU: Gesamtzahl der berücksichtigten Flüge je Intervall ***
total_considered = (
    flight_concepts.groupby(["interval_start", "interval_end"], as_index=False)
    .agg(dominant_runway_count=("aircraft_registration", "nunique"))
)

dominant = dominant.merge(total_considered, on=["interval_start", "interval_end"], how="right")

# ================================
# 6) Lückenloses Intervall-Grid & Merge (auch Nachtruhe-Intervalle)
# ================================
grid = pd.DataFrame({"interval_start": pd.date_range(TIME_START, TIME_STOP, freq="30min")})
grid["interval_end"] = grid["interval_start"] + pd.Timedelta(minutes=30)

out = grid.merge(dominant, on=["interval_start", "interval_end"], how="left")
out["dominant_runway_concept"] = out["dominant_runway_concept"].fillna("")
out["dominant_runway_count"]   = out["dominant_runway_count"].fillna(0).astype(int)

# ================================
# 7) Speichern & Diagnose
# ================================
out = out.sort_values("interval_start").reset_index(drop=True)
out.to_csv(OUT_DOMINANT, index=False, encoding="utf-8")

print(f"✅ Datei gespeichert: {OUT_DOMINANT}")
print(f"Zeilen: {len(out):,} | Zeitraum: {out['interval_start'].min()} – {out['interval_start'].max()}")
print(f"Intervalle ohne Flüge: {(out['dominant_runway_count']==0).sum():,}")


✅ Datei gespeichert: runway_concept_dominant_HALBHOUR.csv
Zeilen: 47,316 | Zeitraum: 2023-01-01 05:50:00 – 2025-09-12 23:20:00
Intervalle ohne Flüge: 12,021


## Kategorisierung METAR Visibility Daten

In [2]:
import re
import numpy as np
import pandas as pd

# =========================
# Dateien
# =========================
FILE_METAR  = "metar_official_HALBHOUR_MATCHED.csv"
OUTPUT_FILE = "visibility_categories_HALBHOUR.csv"

df_metar = pd.read_csv(FILE_METAR)

# =========================
# Zeitspalte / Join-Key erkennen
# =========================
def guess_time_key(df: pd.DataFrame):
    candidates = [
        "time", "timestamp", "timestamp_utc", "datetime", "datetime_utc",
        "interval", "interval_start", "interval_start_utc", "ts", "dt"
    ]
    for c in candidates:
        if c in df.columns:
            return c
    # Fallback: Index
    df["_rowidx_"] = np.arange(len(df))
    return "_rowidx_"

key = guess_time_key(df_metar)

# =========================
# Sichtweite extrahieren
# =========================
def parse_vis_from_text(s: str):
    """Extrahiert Zahlen aus Text wie '8000 metres' oder '10 km'."""
    if not isinstance(s, str):
        return None
    s_low = s.lower()

    km_match = re.search(r'(\d{1,3})\s*km', s_low)
    if km_match:
        return int(km_match.group(1)) * 1000

    m_match = re.search(r'(\d{2,5})\s*m(?:etre|eter|)\b', s_low)
    if m_match:
        return int(m_match.group(1))

    num_match = re.search(r'\b(\d{2,5})\b', s_low)
    if num_match:
        return int(num_match.group(1))

    return None


def extract_visibility(row):
    """Versucht, Sichtweite zu bestimmen – erst vis, dann code."""
    vis_val = None

    # 1) aus 'vis'
    if "vis" in row and pd.notna(row["vis"]):
        vis_val = parse_vis_from_text(str(row["vis"]))

    # 2) falls leer → aus 'code'
    if (vis_val is None) and ("code" in row) and pd.notna(row["code"]):
        code = str(row["code"]).upper()

        if "CAVOK" in code:
            return 10001  # >10 km

        if re.search(r'\b9999\b', code):
            return 10000

        m = re.search(r'\b(\d{4})\b', code)
        if m:
            vis_val = int(m.group(1))

        sm = re.search(r'P?(\d+)\s*SM', code)
        if (vis_val is None) and sm:
            vis_val = int(sm.group(1)) * 1609

    return vis_val


# Sichtweite berechnen
df_metar["vis_m"] = df_metar.apply(extract_visibility, axis=1)
df_metar["vis_m"] = pd.to_numeric(df_metar["vis_m"], errors="coerce").clip(lower=0)

# =========================
# Kategorien bilden (A–H)
# =========================
def vis_category(v):
    if pd.isna(v):
        return np.nan
    v = int(v)
    if 0 <= v <= 200:
        return "A"
    elif v <= 500:
        return "B"
    elif v <= 1000:
        return "C"
    elif v <= 2000:
        return "D"
    elif v <= 4000:
        return "E"
    elif v <= 7000:
        return "F"
    elif v <= 9998:
        return "G"
    else:
        return "H"

df_metar["vis_cat"] = df_metar["vis_m"].apply(vis_category)

# =========================
# Relevante Spalten auswählen & speichern
# =========================
out = df_metar[[key, "vis_m", "vis_cat"]].copy()
out.to_csv(OUTPUT_FILE, index=False)

print("✅ Datei erstellt:", OUTPUT_FILE)
print(out.head())
print(f"ℹ️ {out['vis_m'].isna().sum()} Zeilen ohne ermittelte Sichtweite.")


✅ Datei erstellt: visibility_categories_HALBHOUR.csv
                        time    vis_m vis_cat
0  2023-01-01 05:50:00+00:00  10000.0       H
1  2023-01-01 06:20:00+00:00  10000.0       H
2  2023-01-01 06:50:00+00:00  10000.0       H
3  2023-01-01 07:20:00+00:00  10000.0       H
4  2023-01-01 07:50:00+00:00  10000.0       H
ℹ️ 20 Zeilen ohne ermittelte Sichtweite.


In [3]:
import numpy as np
import pandas as pd

# =========================
# INPUT & OUTPUT
# =========================
# Nimm hier eine Datei, die dein komplettes HALBHOUR-Zeitgitter enthält.
# Falls du nur die Zeitachse willst, kannst du z.B. metar_official_HALBHOUR_MATCHED.csv nehmen.
INPUT_FILE  = "metar_official_HALBHOUR_MATCHED.csv"   # ggf. anpassen
OUTPUT_FILE = "time_features_HALBHOUR.csv"

df = pd.read_csv(INPUT_FILE)

# =========================
# Zeitspalte erkennen & aufbereiten
# =========================
def guess_time_key(df: pd.DataFrame):
    candidates = [
        "Time of Prediction",   # bevorzugt, wenn schon vorhanden
        "time", "timestamp", "timestamp_utc", "datetime", "datetime_utc",
        "interval", "interval_start", "interval_start_utc", "ts", "dt"
    ]
    for c in candidates:
        if c in df.columns:
            return c
    raise RuntimeError("Keine Zeitspalte gefunden. Bitte eine der üblichen Spaltennamen verwenden.")

time_col = guess_time_key(df)

# In datetime konvertieren (ohne harte TZ-Annahme; falls String mit TZ, bleibt sie erhalten)
df[time_col] = pd.to_datetime(df[time_col], errors="coerce")
if df[time_col].isna().any():
    raise ValueError("Mindestens eine Zeitzeile konnte nicht geparst werden. Prüfe das Eingabefile.")

# Für die nachfolgende Logik nennen wir sie wie in deiner Vorlage:
df = df.rename(columns={time_col: "Time of Prediction"})

# =========================
# Diskrete Zeitfeatures
# =========================
final_table = pd.DataFrame()
final_table["Time of Prediction"] = df["Time of Prediction"]

final_table["Hour"]        = final_table["Time of Prediction"].dt.hour
final_table["Minute"]      = final_table["Time of Prediction"].dt.minute
final_table["Day of week"] = final_table["Time of Prediction"].dt.weekday  # Mo=0..So=6
final_table["Month"]       = final_table["Time of Prediction"].dt.month
final_table["Year"]        = final_table["Time of Prediction"].dt.year % 100  # wie Vorlage (0..99)

# =========================
# Zyklisches Encoding (wie Vorlage, 0..1 skaliert)
# =========================
def cyclic_enc(df_, feature, max_val):
    df_[feature + " sin"] = 0.5 * np.sin(2 * np.pi * df_[feature] / max_val) + 0.5
    df_[feature + " cos"] = 0.5 * np.cos(2 * np.pi * df_[feature] / max_val) + 0.5
    return df_

def encode_df(df_):
    df_ = cyclic_enc(df_, "Hour", 24)
    df_ = cyclic_enc(df_, "Minute", 60)
    df_ = cyclic_enc(df_, "Day of week", 7)
    df_ = cyclic_enc(df_, "Month", 12)
    df_ = cyclic_enc(df_, "Year", 100)
    # Originale nach dem Encoding entfernen (Vorlagen-konform)
    df_ = df_.drop(columns=["Hour", "Minute", "Day of week", "Month", "Year"])
    return df_

final_table = encode_df(final_table)

# =========================
# Speichern
# =========================
final_table.to_csv(OUTPUT_FILE, index=False)
print(f"✅ Zeit-Features gespeichert in: {OUTPUT_FILE}")
print(final_table.head())


✅ Zeit-Features gespeichert in: time_features_HALBHOUR.csv
         Time of Prediction  Hour sin  Hour cos  Minute sin  Minute cos  \
0 2023-01-01 05:50:00+00:00  0.982963   0.62941    0.066987        0.75   
1 2023-01-01 06:20:00+00:00  1.000000   0.50000    0.933013        0.25   
2 2023-01-01 06:50:00+00:00  1.000000   0.50000    0.066987        0.75   
3 2023-01-01 07:20:00+00:00  0.982963   0.37059    0.933013        0.25   
4 2023-01-01 07:50:00+00:00  0.982963   0.37059    0.066987        0.75   

   Day of week sin  Day of week cos  Month sin  Month cos  Year sin  Year cos  
0         0.109084         0.811745       0.75   0.933013  0.996057  0.562667  
1         0.109084         0.811745       0.75   0.933013  0.996057  0.562667  
2         0.109084         0.811745       0.75   0.933013  0.996057  0.562667  
3         0.109084         0.811745       0.75   0.933013  0.996057  0.562667  
4         0.109084         0.811745       0.75   0.933013  0.996057  0.562667  


## Erstellung zyklisch encodierter Zeit-Features aus HALBHOUR-Zeitstempeln


In [4]:
import re
import numpy as np
import pandas as pd

# =========================
# INPUT & OUTPUT
# =========================
INPUT_FILE  = "metar_official_HALBHOUR_MATCHED.csv"
OUTPUT_FILE = "wind_direction_HALBHOUR.csv"

df = pd.read_csv(INPUT_FILE)

# =========================
# Zeitspalte erkennen & aufbereiten
# =========================
def guess_time_key(df: pd.DataFrame):
    candidates = [
        "Time of Prediction",   # bevorzugt
        "time", "timestamp", "timestamp_utc", "datetime", "datetime_utc",
        "interval", "interval_start", "interval_start_utc", "ts", "dt"
    ]
    for c in candidates:
        if c in df.columns:
            return c
    raise RuntimeError("Keine Zeitspalte gefunden. Bitte eine der üblichen Spaltennamen verwenden.")

time_col = guess_time_key(df)
df[time_col] = pd.to_datetime(df[time_col], errors="coerce")
if df[time_col].isna().any():
    raise ValueError("Mindestens eine Zeitzeile konnte nicht geparst werden. Prüfe das Eingabefile.")


df = df.rename(columns={time_col: "Time of Prediction"})

# =========================
# Windrichtung extrahieren
# =========================
def parse_wind_dir(val, code_text=None):
    """
    Erwartet z. B. '320 degrees'.
    Fallbacks:
      - 'VRB' → np.nan (keine feste Richtung)
      - '000' → 0 Grad
      - Aus 'code' (METAR)  dddVVKT  (z.B. '32012KT', 'VRB03KT') erkennen.
    """
    # 1) Primär aus wind_dir
    if isinstance(val, str):
        s = val.strip().upper()
        if "VRB" in s:
            return np.nan
        m = re.search(r'\b(\d{1,3})\s*DEGREE', s)  # '320 DEGREES' / '320 degree'
        if m:
            deg = int(m.group(1)) % 360
            return deg
        # reine Zahl?
        m2 = re.search(r'\b(\d{1,3})\b', s)
        if m2:
            return int(m2.group(1)) % 360

    # 2) Fallback: aus METAR code (falls übergeben)
    if isinstance(code_text, str):
        ct = code_text.upper()
        # VRBxxKT
        if re.search(r'\bVRB\d{2,3}KT\b', ct):
            return np.nan
        # dddxxKT (ddd = 000..360)
        m = re.search(r'\b(\d{3})(\d{2,3})KT\b', ct)
        if m:
            deg = int(m.group(1))
            if 0 <= deg <= 360:
                return (0 if deg == 360 else deg)
    return np.nan

# Falls 'code' existiert, mit durchreichen für Fallback
has_code = "code" in df.columns
df["wind_dir_deg"] = [
    parse_wind_dir(row.get("wind_dir", np.nan), row.get("code", None) if has_code else None)
    for _, row in df.iterrows()
]

# =========================
# Zyklisches Encoding (0..1 skaliert, wie bei dir)
# =========================
def cyclic_enc_deg(series_deg, period=360):
    rad = 2 * np.pi * (series_deg % period) / period
    sin_col = 0.5 * np.sin(rad) + 0.5
    cos_col = 0.5 * np.cos(rad) + 0.5
    # NaNs beibehalten
    sin_col[series_deg.isna()] = np.nan
    cos_col[series_deg.isna()] = np.nan
    return sin_col, cos_col

df["wind_dir sin"], df["wind_dir cos"] = cyclic_enc_deg(df["wind_dir_deg"], period=360)

# =========================
# Speichern (nur Zeit + Features)
# =========================
out = df[["Time of Prediction", "wind_dir_deg", "wind_dir sin", "wind_dir cos"]].copy()
out.to_csv(OUTPUT_FILE, index=False)

print(f"✅ Windrichtungs-Features gespeichert in: {OUTPUT_FILE}")
print(out.head())
print(f"ℹ️ {out['wind_dir_deg'].isna().sum()} Zeilen ohne feste Richtung (z. B. VRB).")


✅ Windrichtungs-Features gespeichert in: wind_direction_HALBHOUR.csv
         Time of Prediction  wind_dir_deg  wind_dir sin  wind_dir cos
0 2023-01-01 05:50:00+00:00         220.0      0.178606      0.116978
1 2023-01-01 06:20:00+00:00         240.0      0.066987      0.250000
2 2023-01-01 06:50:00+00:00         230.0      0.116978      0.178606
3 2023-01-01 07:20:00+00:00         240.0      0.066987      0.250000
4 2023-01-01 07:50:00+00:00         240.0      0.066987      0.250000
ℹ️ 13571 Zeilen ohne feste Richtung (z. B. VRB).


## Merge der Daten

In [5]:
import pandas as pd
import numpy as np

# =========================
# Dateipfade
# =========================
FILES = {
    "visibility": "visibility_categories_HALBHOUR.csv",
    "time": "time_features_HALBHOUR.csv",
    "wind": "wind_direction_HALBHOUR.csv",
    "runway": "runway_concept_dominant_HALBHOUR.csv",
}
OUTPUT_FILE = "addons_merged_HALBHOUR.csv"

# =========================
# Helpers
# =========================
def guess_time_key(df: pd.DataFrame):
    candidates = [
        "Time of Prediction","time","timestamp","timestamp_utc",
        "datetime","datetime_utc","interval","interval_start","interval_start_utc"
    ]
    for c in candidates:
        if c in df.columns:
            return c
    raise RuntimeError("Keine Zeitspalte erkannt. Bitte manuell prüfen.")

def to_utc(series: pd.Series) -> pd.Series:
    """Sorgt dafür, dass die Zeitspalte tz-aware in UTC ist."""
    s = pd.to_datetime(series, errors="coerce", utc=False)
    # Wenn schon tz-aware -> nach UTC konvertieren
    if getattr(s.dtype, "tz", None) is not None:
        return s.dt.tz_convert("UTC")
    # Wenn naiv -> als UTC interpretieren (dein Grid ist in UTC)
    return s.dt.tz_localize("UTC")

# =========================
# Dateien einlesen & harmonisieren
# =========================
dfs = {}
for name, path in FILES.items():
    df = pd.read_csv(path)
    key = guess_time_key(df)
    df[key] = to_utc(df[key])
    df = df.rename(columns={key: "Time of Prediction"})
    # Optional: Duplikate pro Zeitstempel entfernen (falls vorhanden)
    if df.duplicated(subset=["Time of Prediction"]).any():
        # Nimm die erste Zeile pro Zeitstempel. Alternativ: aggregieren.
        df = df.sort_values("Time of Prediction").drop_duplicates(subset=["Time of Prediction"], keep="first")
        print(f"ℹ️ Duplikate in '{name}' entfernt (first wins).")
    dfs[name] = df
    tzinfo = getattr(df["Time of Prediction"].dtype, "tz", None)
    print(f"✅ {name} geladen ({len(df)} Zeilen) | dtype: {df['Time of Prediction'].dtype} | tz={tzinfo}")

# =========================
# Mergen aller Add-ons
# =========================
merged = dfs["time"].copy()
for name in ["visibility", "wind", "runway"]:
    before = len(merged)
    merged = merged.merge(dfs[name], on="Time of Prediction", how="left")
    after = len(merged)
    if after != before:
        print(f"⚠️ Hinweis: Merge mit '{name}' hat die Zeilenanzahl verändert ({before} → {after}). "
              f"Prüfe ggf. Duplikate oder Nicht-Eindeutigkeit der Zeitstempel im Add-on.")

# =========================
# Ausgabe
# =========================
merged.to_csv(OUTPUT_FILE, index=False)
print(f"\n✅ Add-ons erfolgreich zusammengeführt: {OUTPUT_FILE}")
print(f"Spalten: {list(merged.columns)}")
print(merged.head())


✅ visibility geladen (47316 Zeilen) | dtype: datetime64[ns, UTC] | tz=UTC
✅ time geladen (47316 Zeilen) | dtype: datetime64[ns, UTC] | tz=UTC
✅ wind geladen (47316 Zeilen) | dtype: datetime64[ns, UTC] | tz=UTC
✅ runway geladen (47316 Zeilen) | dtype: datetime64[ns, UTC] | tz=UTC

✅ Add-ons erfolgreich zusammengeführt: addons_merged_HALBHOUR.csv
Spalten: ['Time of Prediction', 'Hour sin', 'Hour cos', 'Minute sin', 'Minute cos', 'Day of week sin', 'Day of week cos', 'Month sin', 'Month cos', 'Year sin', 'Year cos', 'vis_m', 'vis_cat', 'wind_dir_deg', 'wind_dir sin', 'wind_dir cos', 'interval_end', 'dominant_runway_concept', 'dominant_runway_count']
         Time of Prediction  Hour sin  Hour cos  Minute sin  Minute cos  \
0 2023-01-01 05:50:00+00:00  0.982963   0.62941    0.066987        0.75   
1 2023-01-01 06:20:00+00:00  1.000000   0.50000    0.933013        0.25   
2 2023-01-01 06:50:00+00:00  1.000000   0.50000    0.066987        0.75   
3 2023-01-01 07:20:00+00:00  0.982963   0.370

In [6]:
# Beispiel: Finale Tabelle laden
df = pd.read_csv("addons_merged_HALBHOUR.csv")


df 

,Time of Prediction,Hour sin,Hour cos,Minute sin,Minute cos,Day of week sin,Day of week cos,Month sin,Month cos,Year sin,Year cos,vis_m,vis_cat,wind_dir_deg,wind_dir sin,wind_dir cos,interval_end,dominant_runway_concept,dominant_runway_count
0,2023-01-01 05:50:00+00:00,0.982963,0.629410,0.066987,0.75,0.109084,0.811745,0.75,0.933013,0.996057,0.562667,10000.0,H,220.0,0.178606,0.116978,2023-01-01 06:20:00,EVENING_C1,5
1,2023-01-01 06:20:00+00:00,1.000000,0.500000,0.933013,0.25,0.109084,0.811745,0.75,0.933013,0.996057,0.562667,10000.0,H,240.0,0.066987,0.250000,2023-01-01 06:50:00,EVENING_C1,4
2,2023-01-01 06:50:00+00:00,1.000000,0.500000,0.066987,0.75,0.109084,0.811745,0.75,0.933013,0.996057,0.562667,10000.0,H,230.0,0.116978,0.178606,2023-01-01 07:20:00,NORTHWEST,13
3,2023-01-01 07:20:00+00:00,0.982963,0.370590,0.933013,0.25,0.109084,0.811745,0.75,0.933013,0.996057,0.562667,10000.0,H,240.0,0.066987,0.250000,2023-01-01 07:50:00,NORTHWEST,19
4,2023-01-01 07:50:00+00:00,0.982963,0.370590,0.066987,0.75,0.109084,0.811745,0.75,0.933013,0.996057,0.562667,10000.0,H,240.0,0.066987,0.250000,2023-01-01 08:20:00,NORTHWEST,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47311,2025-09-12 21:20:00+00:00,0.146447,0.853553,0.933013,0.25,0.283058,0.049516,0.00,0.500000,1.000000,0.500000,10000.0,H,NaN,NaN,NaN,2025-09-12 21:50:00,NORTHWEST,24
47312,2025-09-12 21:50:00+00:00,0.146447,0.853553,0.066987,0.75,0.283058,0.049516,0.00,0.500000,1.000000,0.500000,10000.0,H,NaN,NaN,NaN,2025-09-12 22:20:00,NORTHWEST,24
47313,2025-09-12 22:20:00+00:00,0.250000,0.933013,0.933013,0.25,0.283058,0.049516,0.00,0.500000,1.000000,0.500000,10000.0,H,320.0,0.178606,0.883022,2025-09-12 22:50:00,NORTHWEST,18
47314,2025-09-12 22:50:00+00:00,0.250000,0.933013,0.066987,0.75,0.283058,0.049516,0.00,0.500000,1.000000,0.500000,10000.0,H,310.0,0.116978,0.821394,2025-09-12 23:20:00,NORTHWEST,10
